In [1]:
from tdc.single_pred import Epitope
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score, precision_recall_curve
from sklearn.metrics import roc_auc_score, f1_score, average_precision_score, precision_score, recall_score, accuracy_score
from copy import deepcopy
torch.manual_seed(1)
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt  
from sklearn.metrics import roc_curve, auc
from scipy import interp
from sklearn.metrics import roc_auc_score 
from tdc.multi_pred import AntibodyAff
from collections import defaultdict

In [2]:
# data = AntibodyAff(name = 'Protein_SAbDab')
# split = data.get_split()

In [3]:
data =  Epitope(name = 'IEDB_Jespersen')
split = data.get_split()

Found local copy...
Loading...
Done!


In [53]:
train_data = split['train']
valid_data = split['valid']
test_data = split['test']

In [58]:
# def to_protlists(data):

#     protlists = list()
#     temp_dict = data.to_dict("records")
#     for record in temp_dict:
#         protlist = defaultdict(list)
#         antigen = record['Antigen_ID']
#         seq = record['Antigen']
#         desc = ''
#         protlist[antigen].append(desc)
#         protlist[antigen].append(seq)

#         protlists.append(protlist)
        
#     return protlists

In [60]:
lst = train_data['Antigen'].tolist()
maxlen = max([len(A) for A in lst])
maxlen

34350

In [62]:
print([len(A) for A in lst])

[433, 854, 3433, 214, 82, 307, 400, 428, 284, 194, 107, 1291, 108, 307, 3033, 254, 764, 434, 2706, 266, 3390, 82, 685, 357, 3387, 418, 154, 302, 118, 350, 254, 540, 580, 422, 517, 488, 156, 495, 3010, 95, 101, 506, 278, 3010, 133, 507, 1073, 859, 254, 362, 538, 2183, 98, 147, 394, 122, 581, 3011, 846, 3392, 574, 183, 126, 776, 336, 3157, 261, 146, 754, 211, 218, 609, 3430, 797, 1565, 473, 491, 386, 302, 294, 330, 388, 299, 1376, 736, 350, 218, 1732, 153, 1255, 257, 473, 2209, 227, 770, 240, 146, 473, 217, 234, 3432, 391, 64, 3859, 686, 226, 556, 439, 407, 617, 461, 357, 1627, 866, 353, 123, 190, 158, 254, 473, 1935, 3010, 62, 2209, 554, 365, 100, 465, 564, 359, 252, 495, 861, 178, 859, 124, 482, 488, 109, 2332, 261, 124, 770, 332, 401, 204, 340, 2316, 623, 3011, 340, 131, 378, 408, 2193, 665, 531, 992, 209, 475, 158, 160, 256, 94, 2351, 335, 363, 1487, 653, 233, 3011, 585, 180, 368, 142, 479, 866, 711, 623, 708, 626, 245, 100, 352, 103, 222, 115, 727, 183, 277, 3010, 1253, 337, 302, 38

In [5]:
X = 'Antigen'

In [6]:
def data2vocab(data):
	length = len(data)
	vocab_set = set()
	total_length, positive_num = 0, 0
	for i in range(length):
		antigen = data[X][i]
		vocab_set = vocab_set.union(set(antigen))
		Y = data['Y'][i]
		assert len(antigen) > max(Y)
		total_length += len(antigen)
		positive_num += len(Y)
	return vocab_set, positive_num / total_length

In [7]:
train_vocab, train_positive_ratio = data2vocab(train_data)
valid_vocab, valid_positive_ratio = data2vocab(valid_data)
test_vocab, test_positive_ratio = data2vocab(test_data)

In [8]:
vocab_set = train_vocab.union(valid_vocab)
vocab_set = vocab_set.union(test_vocab)
vocab_lst = list(vocab_set)
# logger

In [9]:
def onehot(idx, length):
	lst = [0 for i in range(length)]
	lst[idx] = 1
	return lst 

def zerohot(length):
	return [0 for i in range(length)]

# what is the maxlength here
def standardize_data(data, vocab_lst, maxlength = 300):
	length = len(data)
	standard_data = []
	for i in range(length):
		antigen = data[X][i]
		Y = data['Y'][i] 
		sequence = [onehot(vocab_lst.index(s), len(vocab_lst)) for s in antigen] 
		labels = [0 for i in range(len(antigen))]
		mask = [True for i in range(len(labels))] # labels and mask have the same length
		sequence += (maxlength-len(sequence)) * [zerohot(len(vocab_lst))] #pad to consistent length
		labels += (maxlength-len(labels)) * [0] 
		mask += (maxlength-len(mask)) * [False] # pad to maxlength
		for y in Y:
			labels[y] = 1 		
		sequence, labels, mask = sequence[:maxlength], labels[:maxlength], mask[:maxlength]
		sequence, labels, mask = torch.FloatTensor(sequence), torch.FloatTensor(labels), torch.BoolTensor(mask) 
		# print(sequence.shape, labels.shape, mask.shape)
        # sequence is 2D, labels and mask are 1D
		standard_data.append((sequence, labels, mask))
	return standard_data 

In [22]:
# def standardize_data2(data, vocab_lst):
# 	length = len(data)
# 	standard_data = []
# 	for i in range(length):
# 		antigen = data[X][i]
# 		Y = data['Y'][i] 
# 		sequence = [onehot(vocab_lst.index(s), len(vocab_lst)) for s in antigen] 
# 		labels = [0 for i in range(len(antigen))]
# 		mask = [True for i in range(len(labels))] # labels and mask have the same length
# # 		sequence += (maxlength-len(sequence)) * [zerohot(len(vocab_lst))] #pad to consistent length
# # 		labels += (maxlength-len(labels)) * [0] 
# # 		mask += (maxlength-len(mask)) * [False] # pad to maxlength
# 		for y in Y:
# 			labels[y] = 1 		
# # 		sequence, labels, mask = sequence[:maxlength], labels[:maxlength], mask[:maxlength]
# 		sequence, labels, mask = torch.FloatTensor(sequence), torch.FloatTensor(labels), torch.BoolTensor(mask) 
# 		# print(sequence.shape, labels.shape, mask.shape)
#         # sequence is 2D, labels and mask are 1D
# 		standard_data.append((sequence, labels, mask))
# 	return standard_data 

In [23]:
# train_data_stand2 = standardize_data2(train_data, vocab_lst)
# valid_data_stand2 = standardize_data2(valid_data, vocab_lst)
# test_data_stand2 = standardize_data2(test_data, vocab_lst)

In [10]:
train_data_stand = standardize_data(train_data, vocab_lst)
valid_data_stand = standardize_data(valid_data, vocab_lst)
test_data_stand = standardize_data(test_data, vocab_lst)

In [41]:
train_data_stand2[9][0].shape

torch.Size([194, 24])

In [26]:
class dataset(Dataset):
	def __init__(self, data):
		self.sequences = [i[0] for i in data]
		self.labels = [i[1] for i in data]
		self.mask = [i[2] for i in data] 

	def __getitem__(self, index):
		return self.sequences[index], self.labels[index], self.mask[index]

	def __len__(self):
		return len(self.labels)

In [63]:
train_set = dataset(train_data_stand)
valid_set = dataset(valid_data_stand)
test_set = dataset(test_data_stand)
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
test_loader = DataLoader(test_set, batch_size=16, shuffle=False)

In [95]:
train_loader

TypeError: 'DataLoader' object is not subscriptable

In [94]:
for sequence, labels, mask in train_loader:
    print(sequence)
# for batch, (x,y,z) in enumerate(train_loader):
#         print(batch)

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0., 

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0., 

In [70]:
embedding = nn.Embedding(10, 3)
i = torch.LongTensor([[1,2,4,5],[4,3,2,9]])
o = embedding(i)
i.shape, o.shape

(torch.Size([2, 4]), torch.Size([2, 4, 3]))

In [51]:
type(train_loader)

torch.utils.data.dataloader.DataLoader

In [21]:
# train_data_stand[0][2].shape

torch.Size([300])

In [28]:
def plot(label_lst, predict_lst, name):

    fpr, tpr, thresholds = metrics.roc_curve(label_lst, predict_lst, )
    roc_auc = auc(fpr, tpr)

    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.savefig(name)

In [29]:
class RNN(nn.Module):
    def __init__(self, name, hidden_size, input_size, num_layers = 2):
        super(RNN, self).__init__()
        self.name = name 
        self.hidden_size = hidden_size
        self.input_size = input_size 
        self.rnn = nn.LSTM(         # if use nn.RNN(), it hardly learns
            input_size=input_size,
            hidden_size=hidden_size,         # rnn hidden unit
            num_layers=num_layers,           # number of rnn layer
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )

        self.out = nn.Linear(hidden_size, 1)
        criterion = torch.nn.BCEWithLogitsLoss()  
        self.opt = torch.optim.Adam(self.parameters(), lr=1e-3)
    
    
    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        r_out, (h_n, h_c) = self.rnn(x, None)   # None represents zero initial hidden state

        # choose r_out at the last time step
        out = self.out(r_out)
        out = out.squeeze(-1)
        return out
    
    def learn(self, sequence, labels, mask):
        prediction = self.forward(sequence)
        # print("size", prediction.shape, labels.shape, mask.shape)
        criterion = torch.nn.BCEWithLogitsLoss(size_average=True, weight = mask)  
        loss = criterion(prediction, labels)
        self.opt.zero_grad() 
        loss.backward() 
        self.opt.step()
        
    def test(self, test_loader, name):
        label_lst, prediction_lst = [], []
        for sequence, labels, mask in test_loader:
            prediction = self.forward(sequence)
            prediction = torch.sigmoid(prediction)
            for pred, label, msk in zip(prediction, labels, mask):
                num = sum(msk.tolist()) 
                pred = pred.tolist()[:num] 
                label = label.tolist()[:num] 
                label_lst.extend(label)
                prediction_lst.extend(pred)
        sort_pred = deepcopy(prediction_lst)
        sort_pred.sort() 
        threshold = sort_pred[int(len(sort_pred)*0.9)]
        float2binary = lambda x:0 if x<threshold else 1
        binary_pred_lst = list(map(float2binary, prediction_lst))
        plot(label_lst, prediction_lst, name)
        print('roc_auc', roc_auc_score(label_lst, prediction_lst), 
    		  'F1', f1_score(label_lst, binary_pred_lst), 
    		  'prauc', average_precision_score(label_lst, binary_pred_lst))
        

In [42]:
model = RNN(name = 'Epitope', hidden_size=100, input_size=len(vocab_lst))
epoch = 10
for ep in range(epoch):
    for sequence, labels, mask in train_loader:
        print(sequence.shape, labels.shape, mask.shape)
        model.learn(sequence, labels, mask)
    model.test(test_loader, name = model.name + '_' + str(ep) + '.png')

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 2351 and 96 in dimension 1 at /pytorch/aten/src/TH/generic/THTensor.cpp:612